# Day 04: Repose Record
[link](https://adventofcode.com/2018/day/4)

In [1]:
from datetime import datetime
import re
import pandas as pd
import numpy as np

columns = ['guard_id', 'timestamp', 'minute']

def parse_timestamp(text):
  y,mo,d,h,mi = map(int, re.findall(r'\d+', text))
  return mi + d*100 + mo*10000 + y*1000000

column_data_types = {
  'guard_id': np.uint16,
  'timestamp': np.uint32,
  'minute': np.uint8
} 

def range_from_timestamp(guard_id, start, end):
  return pd.DataFrame([(guard_id, i, i%100) for i in range(start, end)], columns=columns).astype(dtype=column_data_types)

def parse_guard_ranges(log):
  guard_id, asleep_since = None, None
  guard_ranges = pd.DataFrame(columns=columns)
  for line in sorted(log):
    timestamp_raw, event = line.split('] ')
    if event.startswith('Guard'):
      m = re.match(r'Guard #(\d+) begins shift', event)
      guard_id, asleep_since = int(m.group(1)), None
    else:
      timestamp = parse_timestamp(timestamp_raw[1:])
      if event.startswith('falls asleep'):
        asleep_since = timestamp
      elif event.startswith('wakes up'):
        assert (guard_id is not None) and (asleep_since is not None), f'{line}'
        sleep_range = range_from_timestamp(guard_id, asleep_since, timestamp)
        guard_ranges = pd.concat([guard_ranges, sleep_range], ignore_index=True)
        asleep_since = None
  return guard_ranges

def guard_minute(log):
  guard_ranges = parse_guard_ranges(log)
  minutes_per_guard = guard_ranges.guard_id.value_counts().head(3)
  selected_guard_id = minutes_per_guard.index[0]
  display(minutes_per_guard)
  
  guard_minutes = guard_ranges[guard_ranges.guard_id == selected_guard_id].minute.value_counts().head(3)
  selected_minute = guard_minutes.index[0]
  display(guard_minutes)
  assert guard_minutes.values[0] > guard_minutes.values[1]
  print(f'Selected guard id: {selected_guard_id}, selected minute: {selected_minute}')
  return selected_guard_id * selected_minute

In [2]:
test_in = [
  '[1518-11-05 00:45] falls asleep',
  '[1518-11-01 00:00] Guard #10 begins shift',
  '[1518-11-01 00:05] falls asleep',
  '[1518-11-01 00:25] wakes up',
  '[1518-11-03 00:29] wakes up',
  '[1518-11-01 00:30] falls asleep',
  '[1518-11-01 00:55] wakes up',
  '[1518-11-01 23:58] Guard #99 begins shift',
  '[1518-11-02 00:40] falls asleep',
  '[1518-11-02 00:50] wakes up',
  '[1518-11-03 00:05] Guard #10 begins shift',
  '[1518-11-03 00:24] falls asleep',
  '[1518-11-04 00:02] Guard #99 begins shift',
  '[1518-11-04 00:36] falls asleep',
  '[1518-11-04 00:46] wakes up',
  '[1518-11-05 00:03] Guard #99 begins shift',
  '[1518-11-05 00:55] wakes up'
]

expected_result = 240
actual_result = guard_minute(test_in)
assert expected_result == actual_result, f'Expected {expected_result}, got {actual_result}'

10    50
99    30
Name: guard_id, dtype: int64

24    2
54    1
28    1
Name: minute, dtype: int64

Selected guard id: 10, selected minute: 24


In [3]:
with open('04 input.txt', 'r') as file:
  puzzle_input = [line for line in file]

guard_minute(puzzle_input)

1523    483
1607    480
2707    457
Name: guard_id, dtype: int64

43    14
42    13
41    13
Name: minute, dtype: int64

Selected guard id: 1523, selected minute: 43


65489

**Part 1 right answer:** `65489`

## Part 2: Which guard is most frequently asleep on the same minute?

In [4]:
def frequently_asleep(log):
  guard_ranges = parse_guard_ranges(log)
  groups = guard_ranges.groupby(['guard_id', 'minute'])\
    .size()\
    .sort_values(ascending=False)\
    .head(3)
  assert groups.values[0] > groups.values[1]
  display(groups)
  selected_id, selected_minute = groups.index[0]
  print(f'Guard with id {selected_id} was most frequently asleep on the minute {selected_minute}')
  return selected_id * selected_minute

In [5]:
expected_result_2 = 4455
actual_result_2 = frequently_asleep(test_in)
assert expected_result_2 == actual_result_2, f'Expected {expected_result_2}, got {actual_result_2}'

guard_id  minute
99        45        3
          42        2
          49        2
dtype: int64

Guard with id 99 was most frequently asleep on the minute 45


In [6]:
frequently_asleep(puzzle_input)

guard_id  minute
107       36        16
1607      34        15
          33        15
dtype: int64

Guard with id 107 was most frequently asleep on the minute 36


3852

**Part 2 right answer:** `3852`